In [1]:
from numpy import load
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.nn import Conv1d
from torch.nn import ReLU
from torch.optim import Adam
from sklearn.model_selection import train_test_split


In [3]:
data_real = load('/kaggle/input/footprints/real.npz')
data_fake = load('/kaggle/input/footprints/fake.npz')

features_real = data_real["x"]
features_fake = data_fake["x"]

labels_real = data_real["y"]
labels_fake = data_fake["y"]

features = np.concatenate((features_fake, features_real), axis=0)
labels = np.concatenate((labels_fake, labels_real), axis=0)

features = np.where(np.isfinite(features), features, -10)
X_train, X_val, Y_train, Y_val = train_test_split(features, labels, test_size = 0.2, shuffle = True)

In [4]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y):
    self.tensor_y = torch.tensor(y)
    self.tensor_x = torch.tensor(x)
    

  def __len__(self):
    return len(self.tensor_x)
  
  def __getitem__(self, idx):
    return self.tensor_x[idx], self.tensor_y[idx]

In [5]:
train_dataset = Dataset(X_train, Y_train)
test_dataset = Dataset(X_val, Y_val)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
class Model(nn.Module):
    def __init__(self, n_classes=2, hidden_size=256, input_dim=97):
        super(Model, self).__init__()
        self.conv = Conv1d(in_channels=1998, out_channels=496, kernel_size=5, stride=1)
        self.relu = ReLU()
        
        self.lstm = nn.LSTM(input_dim, hidden_size, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_size, n_classes, bias=True)
        self.sig = nn.Sigmoid()
     
    def forward(self,x):
        x = self.conv(x)
        x = self.relu(x)        
        
        x = self.lstm(x)
        x = self.linear(x[0])
        x = self.sig(x)
        
        return x.float()

In [33]:
INIT_LR = 0.001
EPOCHS = 20
model = Model()
opt = Adam(model.parameters(), lr = INIT_LR)
criterion = nn.CrossEntropyLoss()
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()

In [34]:
if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()
model.train()
for e in range(0, EPOCHS):
	totalTrainLoss = 0 # to calculate loss
	trainCorrect = 0 # to calculate accuracy

	trainDataLoader = torch.utils.data.DataLoader(train_dataset, BATCH_SIZE)
	n_samples=0
	for x,y in tqdm(trainDataLoader):

		x = torch.tensor(x.float(), device=device)
		y = torch.tensor(y.float(), device=device)

		pred = model(x)

		y = y.long()
		loss = criterion(pred.view(-1, pred.size(2)), y.view(-1))

		totalTrainLoss += loss
		n_samples+=y.shape[0]*y.shape[1]
		_,predicted=torch.max(pred,2)
		
		trainCorrect += (predicted.view(-1)==y.view(-1)).sum().item()
		opt.zero_grad()
		loss.backward()
		nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
		opt.step()
		
		
	print("loss = ", totalTrainLoss/len(train_dataset))
	accurcy=100*trainCorrect/(n_samples)
	print(accurcy)

  0%|          | 0/38 [00:00<?, ?it/s]/tmp/ipykernel_34/285787563.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x.float(), device=device)
/tmp/ipykernel_34/285787563.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y.float(), device=device)
100%|██████████| 38/38 [00:01<00:00, 25.69it/s]


loss =  tensor(0.0134, device='cuda:0', grad_fn=<DivBackward0>)
88.69405241935483


100%|██████████| 38/38 [00:01<00:00, 26.38it/s]


loss =  tensor(0.0131, device='cuda:0', grad_fn=<DivBackward0>)
89.98689516129032


100%|██████████| 38/38 [00:01<00:00, 26.43it/s]


...

loss =  tensor(0.0131, device='cuda:0', grad_fn=<DivBackward0>)
89.98689516129032


100%|██████████| 38/38 [00:01<00:00, 25.58it/s]


loss =  tensor(0.0131, device='cuda:0', grad_fn=<DivBackward0>)
89.98689516129032


100%|██████████| 38/38 [00:01<00:00, 25.16it/s]

loss =  tensor(0.0131, device='cuda:0', grad_fn=<DivBackward0>)
89.98689516129032


In [30]:
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

total_acc_test = 0
total_samples = 0

with torch.no_grad():
  for test_input, test_label in tqdm(test_dataloader):

    test_input = torch.tensor(test_input.float(), device=device)
    output = torch.tensor(test_label.float(), device=device)

    output=output.long()
    pred = model(test_input) 

    _,predicted=torch.max(pred,2)
    total_acc_test += (predicted.view(-1)==output.view(-1)).sum().item()

    total_samples += test_label.shape[0]*test_label.shape[1]  # Update total samples count
    

  total_acc_test =100*total_acc_test/total_samples

print(f'\nTest Accuracy: {total_acc_test}')


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_34/1003688201.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_input = torch.tensor(test_input.float(), device=device)
/tmp/ipykernel_34/1003688201.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = torch.tensor(test_label.float(), device=device)
100%|██████████| 10/10 [00:00<00:00, 44.65it/s]


Test Accuracy: 93.90591397849462
